# Holmberg II X-1

In [3]:
import time
import glob
import importlib
import tempfile
import numpy as np
import xarray as xr
import astropy
import astropy.io.fits as fits
from astropy.table import Table
from astropy.wcs import WCS

from astro import astroutils
from astro import mastrometry
from astro import mimage
from astro import mphot

from utils import mutils
from utils import mlogging 

from astro.astroutils import jd2UTCtimestamp, UTCtimestamp2jd

from constants import dr_tek2k, plate_scales

log = mlogging.getLogger('photometry')

%matplotlib ipympl

In [4]:
for x in plate_scales.values():
    dr = dr_tek2k * .3252/ x
    bw = 7 * .3252/ x
    sa = 15 * .3252/ x
    print(f'{dr} {bw} {sa}')

3.8 6.999999999999999 15.0
6.08 11.2 24.000000000000004
2.8085454545454547 5.173636363636363 11.086363636363636
2.0596 3.7939999999999996 8.13


In [127]:
comps = [     
            ('08 19 35.27861', '70 42 46.68284'), # A=1774 SNR=592   stdev = 0.2 (previously #1)  
            ('08 19 42.614',   '70 40 23.9842'),    # A=4800 SNR=1078 stdev = 0.05 (previously #5)
            ('08 19 57.87424', '70 42 53.07892'),   # A=2295 SNR=775 stdev = 0.03 (previously #6)
            ('08 19 33.91390', '70 44 27.99391'),   # A=3500 SNR=904      
            ('08 19 55.25860', '70 42 08.06211'),  #
        ]
for c in comps:
    print(mphot.equatorial2pix(c, '/home/canis/data/cameras/tek2k/seeing_below_2.0/g21d018.050.a.nc'))

(array(959.44055901), array(1456.47867672))
(array(1400.13872861), array(1560.64708665))
(array(945.1374928), array(1801.1145993))
(array(647.52730314), array(1441.00439609))
(array(1083.00955791), array(1759.01404121))


## aperphot

In [138]:
importlib.reload(mphot)

<module 'astro.mphot' from '/home/canis/python/astro/mphot.py'>

In [165]:
import time
def trust_sleep(n):
    print(f'Sleeping for {n} seconds')
    start = time.time()
    while time.time() - start < n:
        time.sleep(n-(time.time()-start))

In [161]:
trust_sleep(10)

Sleeping for 10 seconds


In [182]:
from astro import mphot
from astro import mimage
import importlib
importlib.reload(mphot)
importlib.reload(mimage)
import glob

target= ('08 19 28.99', '70 42 19.4') # SIMBAD values is correct
comps = [     
            ('08 19 35.27861', '70 42 46.68284'), # A=1774 SNR=592   stdev = 0.2 (previously #1)  
            ('08 19 42.614',   '70 40 23.9842'),    # A=4800 SNR=1078 stdev = 0.05 (previously #5)
            ('08 19 57.87424', '70 42 53.07892'),   # A=2295 SNR=775 stdev = 0.03 (previously #6)
            ('08 19 33.91390', '70 44 27.99391'),   # A=3500 SNR=904      
            ('08 19 15.57437', '70 44 42.37452'),   # A=2014 SNR=637.7
        ]

saturations = {'tek2k': 28000, 'nd12': 28000, 'Marana': 56000, 'acam': 56000}
plate_scales = {'tek2k': .3252, 'nd12': .20325, 'Marana': .44, 'acam': 0.6}
dr_tek2k = 3.8
version='_v3'

def save_cntrds(cntrds, fn):
    foo = cntrds.copy()
    foo.attrs['compcoords'] = str(foo.compcoords)
    foo.to_netcdf(fn, mode='w')
print(f'{len(comps)} comps')

#trust_sleep(2*86400)
#trust_sleep(90000)

scope = {'tek2k' : 61, 'nd12': 61, 'Marana': 13, 'acam': 13}
for i in [2.0, 2.5, 3.0, 3.5, 4.0, 6.0]:
    for c in ['tek2k', 'nd12', 'Marana', 'acam']: # 'tek2k', 'nd12', 'Marana'
        outpath = f'{HOME}/data/centroids/{c}/seeing_below_{i}/cntrds{version}.nc'                
        if os.path.exists(outpath):
            log.info(f'{i} {c} {n} already done')
            continue

        dr = dr_tek2k * plate_scales['tek2k'] / plate_scales[c]
        bw = 7 * plate_scales['tek2k'] / plate_scales[c]
        sa = 15 * plate_scales['tek2k'] / plate_scales[c]

        log.info(f'{c} {i}')
        log.info(f'Using saturation={saturations[c]}')
        log.info(f'Using disk radius ={dr} {bw} {sa}')
        log.info(f'Using {len(comps)} comps')

        fdir = f'{HOME}/data/cameras/{c}/seeing_below_{i}'
        flist = sorted(glob.glob(f'{fdir}/*.a.nc'))
        try:
            cntrds = mphot.aperphot(flist, startcoords=target,
                                compcoords=comps, ncompmax=5, snrmin=100,
                                diskradius=dr, bufferwidth=bw, skyannulus=sa,
                                Nthresh=2.4, autoradius='fixed', emax=0.5,
                                saturation=saturations[c],
                                nhist=1, quick='noctrd', maxcpus=None, verbose=True)
            save_cntrds(cntrds, outpath)
        except Exception as e:
            print(e)
            print('HEY')


15:03:08 Holmberg   INFO   : tek2k 2.0
15:03:08 Holmberg   INFO   : Using saturation=28000
15:03:08 Holmberg   INFO   : Using disk radius =3.8 6.999999999999999 15.0
15:03:08 Holmberg   INFO   : Using 5 comps
15:03:08 mphot      INFO   : Using astrometric solutions for coordinates.
15:03:08 mphot      INFO   : g20d340.025.a.nc
15:03:08 mphot      INFO   :   UTC of first frame = 07:15:40.786


5 comps


15:03:08 mphot      INFO   : g20d340.026.a.nc
15:03:08 mphot      INFO   :   UTC of first frame = 07:27:11.207
15:03:09 mphot      INFO   : g20d340.027.a.nc
15:03:09 mphot      INFO   :   UTC of first frame = 07:38:41.693
15:03:10 mphot      INFO   : g20d340.028.a.nc
15:03:10 mphot      INFO   :   UTC of first frame = 07:50:12.277
15:03:10 mphot      INFO   : g20d340.029.a.nc
15:03:10 mphot      INFO   :   UTC of first frame = 08:01:43.013
15:03:11 mphot      INFO   : g20d340.030.a.nc
15:03:11 mphot      INFO   :   UTC of first frame = 08:13:13.583
15:03:11 mphot      INFO   : g20d340.031.a.nc
15:03:11 mphot      INFO   :   UTC of first frame = 08:24:44.153
15:03:12 mphot      INFO   : g20d340.032.a.nc
15:03:12 mphot      INFO   :   UTC of first frame = 08:36:14.537
15:03:13 mphot      INFO   : g20d340.033.a.nc
15:03:13 mphot      INFO   :   UTC of first frame = 08:47:44.852
15:03:13 mphot      INFO   : g20d340.034.a.nc
15:03:13 mphot      INFO   :   UTC of first frame = 08:59:15.373
1

15:03:49 mphot      INFO   : g20d342.036.a.nc
15:03:49 mphot      INFO   :   UTC of first frame = 09:14:38.548
15:03:50 mphot      INFO   : g20d342.037.a.nc
15:03:50 mphot      INFO   :   UTC of first frame = 09:26:08.985
15:03:50 mphot      INFO   : g20d342.038.a.nc
15:03:50 mphot      INFO   :   UTC of first frame = 09:37:39.401
15:03:51 mphot      INFO   : g20d342.039.a.nc
15:03:51 mphot      INFO   :   UTC of first frame = 09:49:09.835
15:03:51 mphot      INFO   : g20d342.040.a.nc
15:03:51 mphot      INFO   :   UTC of first frame = 10:00:40.362
15:03:52 mphot      INFO   : g20d342.041.a.nc
15:03:52 mphot      INFO   :   UTC of first frame = 10:12:10.675
15:03:52 mphot      INFO   : g20d342.042.a.nc
15:03:52 mphot      INFO   :   UTC of first frame = 10:23:41.169
15:03:53 mphot      INFO   : g20d342.043.a.nc
15:03:53 mphot      INFO   :   UTC of first frame = 10:35:11.880
15:03:53 mphot      INFO   : g20d342.044.a.nc
15:03:54 mphot      INFO   :   UTC of first frame = 10:46:42.216
1

15:04:30 mphot      INFO   : g20d357.033.a.nc
15:04:30 mphot      INFO   :   UTC of first frame = 07:53:16.950
15:04:31 mphot      INFO   : g20d357.034.a.nc
15:04:31 mphot      INFO   :   UTC of first frame = 08:04:47.509
15:04:31 mphot      INFO   : g20d357.035.a.nc
15:04:32 mphot      INFO   :   UTC of first frame = 08:16:17.999
15:04:32 mphot      INFO   : g20d357.036.a.nc
15:04:32 mphot      INFO   :   UTC of first frame = 08:27:48.449
15:04:33 mphot      INFO   : g20d357.037.a.nc
15:04:33 mphot      INFO   :   UTC of first frame = 08:39:18.944
15:04:33 mphot      INFO   : g20d357.038.a.nc
15:04:33 mphot      INFO   :   UTC of first frame = 08:50:49.589
15:04:34 mphot      INFO   : g20d357.039.a.nc
15:04:34 mphot      INFO   :   UTC of first frame = 09:02:20.059
15:04:34 mphot      INFO   : g20d357.040.a.nc
15:04:34 mphot      INFO   :   UTC of first frame = 09:13:50.558
15:04:35 mphot      INFO   : g20d357.041.a.nc
15:04:35 mphot      INFO   :   UTC of first frame = 09:25:21.094
1

...

19:09:24 mphot      INFO   :   UTC of first frame = 03:53:03.376
19:09:25 mphot      INFO   : g19d148.028.a.nc
19:09:25 mphot      INFO   :   UTC of first frame = 04:01:26.376
19:09:26 mphot      INFO   : g19d148.029.a.nc
19:09:26 mphot      INFO   :   UTC of first frame = 04:09:49.776
19:09:27 mphot      INFO   : g19d148.031.a.nc
19:09:27 mphot      INFO   :   UTC of first frame = 04:20:47.326
19:09:27 mphot      INFO   : g19d148.032.a.nc
19:09:27 mphot      INFO   :   UTC of first frame = 04:36:41.726
19:09:28 mphot      INFO   : g19d148.033.a.nc
19:09:28 mphot      INFO   :   UTC of first frame = 04:52:37.276
19:09:29 mphot      INFO   : g20d032.022.a.nc
19:09:29 mphot      INFO   :   UTC of first frame = 02:17:14.946
19:09:30 mphot      INFO   : g20d032.023.a.nc
19:09:30 mphot      INFO   :   UTC of first frame = 02:28:03.046
19:09:30 mphot      INFO   : g20d032.024.a.nc
19:09:31 mphot      INFO   :   UTC of first frame = 02:38:51.096
19:09:31 mphot      INFO   : g20d032.025.a.nc
1

19:10:21 mphot      INFO   :   UTC of first frame = 07:36:05.196
19:10:22 mphot      INFO   : g20d033.063.a.nc
19:10:22 mphot      INFO   :   UTC of first frame = 07:46:53.296
19:10:23 mphot      INFO   : g20d033.064.a.nc
19:10:23 mphot      INFO   :   UTC of first frame = 07:57:41.296
19:10:23 mphot      INFO   : g20d033.065.a.nc
19:10:24 mphot      INFO   :   UTC of first frame = 08:08:29.946
19:10:24 mphot      INFO   : g20d033.066.a.nc
19:10:24 mphot      INFO   :   UTC of first frame = 08:19:18.196
19:10:25 mphot      INFO   : g20d033.067.a.nc
19:10:25 mphot      INFO   :   UTC of first frame = 08:30:06.096
19:10:26 mphot      INFO   : g20d033.068.a.nc
19:10:26 mphot      INFO   :   UTC of first frame = 08:40:54.196
19:10:27 mphot      INFO   : g20d033.069.a.nc
19:10:27 mphot      INFO   :   UTC of first frame = 08:51:42.346
19:10:27 mphot      INFO   : g20d033.070.a.nc
19:10:28 mphot      INFO   :   UTC of first frame = 09:02:30.546
19:10:28 mphot      INFO   : g20d033.071.a.nc
1

19:10:46 mphot      ERROR  : failed centroid! starnum=2 t=72125838.06 x0=2743.72 y0=2244.82
19:10:46 mphot      ERROR  : failed centroid! starnum=3 t=72125838.06 x0=2497.72 y0=2373.27
19:10:46 mphot      ERROR  : failed centroid! starnum=4 t=72125838.06 x0=2336.30 y0=2177.70
19:10:46 mphot      ERROR  : failed centroid! starnum=5 t=72125838.06 x0=2309.53 y0=2026.98
19:10:46 mphot      INFO   : g20d039.034.a.nc
19:10:47 mphot      INFO   :   UTC of first frame = 04:16:22.547
19:10:47 mphot      ERROR  : failed centroid! starnum=0 t=72126486.11 x0=2546.69 y0=2135.23
19:10:47 mphot      ERROR  : failed centroid! starnum=1 t=72126486.11 x0=2502.39 y0=2187.56
19:10:47 mphot      ERROR  : failed centroid! starnum=2 t=72126486.11 x0=2739.58 y0=2246.09
19:10:47 mphot      ERROR  : failed centroid! starnum=3 t=72126486.11 x0=2494.86 y0=2373.98
19:10:47 mphot      ERROR  : failed centroid! starnum=4 t=72126486.11 x0=2334.30 y0=2178.04
19:10:47 mphot      ERROR  : failed centroid! starnum=5 t=721

19:11:06 mphot      INFO   : g20d039.063.a.nc
19:11:06 mphot      INFO   :   UTC of first frame = 09:35:49.047
19:11:07 mphot      INFO   : g20d039.064.a.nc
19:11:07 mphot      INFO   :   UTC of first frame = 09:46:36.997
19:11:08 mphot      INFO   : g20d039.065.a.nc
19:11:08 mphot      INFO   :   UTC of first frame = 09:57:25.247
19:11:09 mphot      INFO   : g20d039.066.a.nc
19:11:09 mphot      INFO   :   UTC of first frame = 10:08:13.197
19:11:09 mphot      INFO   : g20d039.067.a.nc
19:11:10 mphot      INFO   :   UTC of first frame = 10:19:02.747
19:11:10 mphot      INFO   : g20d039.068.a.nc
19:11:11 mphot      INFO   :   UTC of first frame = 10:29:50.897
19:11:11 mphot      INFO   : g20d039.069.a.nc
19:11:11 mphot      INFO   :   UTC of first frame = 10:40:38.947
19:11:12 mphot      INFO   : g20d039.070.a.nc
19:11:12 mphot      INFO   :   UTC of first frame = 10:51:27.047
19:11:13 mphot      INFO   : g20d039.071.a.nc
19:11:13 mphot      INFO   :   UTC of first frame = 11:02:15.347
1

19:12:08 mphot      INFO   : g20d146.027.a.nc
19:12:08 mphot      INFO   :   UTC of first frame = 04:05:49.360
19:12:09 mphot      INFO   : g20d146.028.a.nc
19:12:09 mphot      INFO   :   UTC of first frame = 04:21:37.860
19:12:10 mphot      INFO   : g20d146.029.a.nc
19:12:10 mphot      INFO   :   UTC of first frame = 04:37:26.210
19:12:10 mphot      INFO   : g20d146.030.a.nc
19:12:11 mphot      INFO   :   UTC of first frame = 04:53:14.910
19:12:11 mphot      INFO   : g20d147.035.a.nc
19:12:12 mphot      INFO   :   UTC of first frame = 03:38:21.610
19:12:12 mphot      INFO   : g20d147.036.a.nc
19:12:12 mphot      INFO   :   UTC of first frame = 03:54:10.110
19:12:13 mphot      INFO   : g20d147.037.a.nc
19:12:13 mphot      INFO   :   UTC of first frame = 04:09:58.360
19:12:14 mphot      INFO   : g20d147.038.a.nc
19:12:14 mphot      INFO   :   UTC of first frame = 04:25:47.810
19:12:15 mphot      INFO   : g20d147.039.a.nc
19:12:15 mphot      INFO   :   UTC of first frame = 04:41:36.510
1

## aperphot: compstars

In [180]:
print(len(comps))
for i in [3.0]:
    for n in range(len(comps)):
        for c in ['tek2k', 'nd12', 'Marana', 'acam']:
#             if c=='acam' and i==2.0:
#                 continue
            outpath = f'{HOME}/data/centroids/{c}/seeing_below_{i}/cntrds{version}_{n+1}.nc'

            if os.path.exists(outpath):
                log.info(f'{i} {c} {n+1} already done')
                continue

            dr = dr_tek2k * plate_scales['tek2k'] / plate_scales[c]
            bw = 7 * plate_scales['tek2k'] / plate_scales[c]
            sa = 15 * plate_scales['tek2k'] / plate_scales[c]

            log.info(f'{c} {i} {n+1}')
            log.info(f'Using saturation={saturations[c]}')
            log.info(f'Using disk radius ={dr} {bw} {sa}')

            fdir = f'{HOME}/data/cameras/{c}/seeing_below_{i}'
            flist = sorted(glob.glob(f'{fdir}/*.a.nc'))

            if n == 0:
                used_comps = comps[1:]
            elif n == len(comps) - 1:
                used_comps =  comps[:n]
            else:
                used_comps = list(np.concatenate([comps[:n], comps[n+1:]])) # avoid truth error
            try:
                cntrds = mphot.aperphot(flist, startcoords=target,
                                    compcoords=comps, ncompmax=5, snrmin=100,
                                    diskradius=dr, bufferwidth=bw, skyannulus=sa,
                                    Nthresh=2.4, autoradius='fixed', emax=0.5,
                                    saturation=saturations[c],
                                    nhist=1, quick='noctrd', maxcpus=None, verbose=True)
                save_cntrds(cntrds, outpath)
            except Exception as e:
                print(e)
                print('HEY')

14:53:26 Holmberg   INFO   : 3.0 tek2k 1 already done
14:53:26 Holmberg   INFO   : 3.0 nd12 1 already done
14:53:26 Holmberg   INFO   : 3.0 Marana 1 already done
14:53:26 Holmberg   INFO   : 3.0 acam 1 already done
14:53:26 Holmberg   INFO   : 3.0 tek2k 2 already done
14:53:26 Holmberg   INFO   : 3.0 nd12 2 already done
14:53:26 Holmberg   INFO   : 3.0 Marana 2 already done
14:53:26 Holmberg   INFO   : 3.0 acam 2 already done
14:53:26 Holmberg   INFO   : 3.0 tek2k 3 already done
14:53:26 Holmberg   INFO   : 3.0 nd12 3 already done
14:53:26 Holmberg   INFO   : 3.0 Marana 3 already done
14:53:26 Holmberg   INFO   : 3.0 acam 3 already done
14:53:26 Holmberg   INFO   : 3.0 tek2k 4 already done
14:53:26 Holmberg   INFO   : 3.0 nd12 4 already done
14:53:27 Holmberg   INFO   : 3.0 Marana 4 already done
14:53:27 Holmberg   INFO   : 3.0 acam 4 already done
14:53:27 Holmberg   INFO   : 3.0 tek2k 5 already done
14:53:27 Holmberg   INFO   : 3.0 nd12 5 already done
14:53:27 Holmberg   INFO   : 3.0 

5
